# Warp a disk

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from tqdm.auto import tqdm

import diskwarp
from diskwarp import helper

import astropy.constants as c
import astropy.units as u

au = c.au.cgs.value
year = (1. * u.year).cgs

%matplotlib widget
%config InlineBackend.figure_format = 'retina'

## General Definitions of disk surface and warp parameters

Calculate the interfaces and cell centers of the disk surface

In [ ]:
r_i = np.linspace(0.1, 100, 201) * au
r_c = 0.5 * (r_i[1:] + r_i[:-1])

In [ ]:
M_star = c.M_sun.cgs.value

In [ ]:
surf     = helper.get_surface(r_i, nphi=100, z0=0.03 * au, r0=au, r_taper=150 * au, q_taper=2)
p0_i     = surf['points_i']
p0_c     = surf['points_c']
ri       = surf['ri']
rc       = surf['rc']
phic     = surf['phic']
phii     = surf['phii']
nr, nphi = p0_c.shape[:-1]

In [ ]:
f, ax = plt.subplots()
ax.plot(rc[:, 0] / au, p0_c[:, 0, -1] / rc[:, 0])

Define the warp (inclination for each ring): logistic function from $i=0$ outside transitioning to `warp_max` at radius `r0` over a transition width of `dr`.

In [ ]:
warp_c = helper.warp(r_c, i_in=45, i_out=20.0, r0=75. * au, dr=10.0 * au)
warp_i = helper.warp(r_i, i_in=45, i_out=20.0, r0=75. * au, dr=10.0 * au)

Define each radius' twist

In [ ]:
twist_i = helper.twist(r_i, PA_in=20.0, PA_out=0., r0=75. * au, dr=10.0 * au)
twist_c = helper.twist(r_c, PA_in=20.0, PA_out=0., r0=75. * au, dr=10.0 * au)

Calculate the velocities of all centers and edges: we assume perfect keplerian rotation, so
$v_\phi \propto r^{-1/2} \propto \left(x^2 + y^2\right)^{-1/4}$

For testing, we might take the radial gradient out and just set $v_\phi = 1$.

Here, `v0_c` and `v0_i` contain the x-, y-, and z-component of the velocity.

In [ ]:
_rc = np.sqrt(p0_c[:, :, 0]**2 + p0_c[:, :, 1]**2)
_ri = np.sqrt(p0_i[:, :, 0]**2 + p0_i[:, :, 1]**2)
vk_c = np.sqrt(c.G.cgs.value * M_star / _rc)
vk_i = np.sqrt(c.G.cgs.value * M_star / _ri)

#vk_c = 0.0 * vk_c + 1.0  ##### make it uniform ######
#vk_i = 0.0 * vk_i + 1.0  ##### make it uniform ######

In [ ]:
v0_c = vk_c[None, :, :] * np.array([-np.sin(phic), np.cos(phic), np.zeros_like(phic)])
v0_i = vk_i[None, :, :] * np.array([-np.sin(phii), np.cos(phii), np.zeros_like(phii)])

v0_c = np.moveaxis(v0_c, 0, 2)
v0_i = np.moveaxis(v0_i, 0, 2)

## Apply the warp/twist

In [ ]:
p1_c = diskwarp.fmodule.apply_matrix2d(p0_c, warp_c, twist_c)
v1_c = diskwarp.fmodule.apply_matrix2d(v0_c, warp_c, twist_c)
p1_i = diskwarp.fmodule.apply_matrix2d(p0_i, warp_i, twist_i)
v1_i = diskwarp.fmodule.apply_matrix2d(v0_i, warp_i, twist_i)

In [ ]:
ir = np.array([50, 75, 100, 125, 150])
s_phi = 4

scale = 0.2 * r_c[ir] / au / np.sqrt((v1_c[ir, :, :]**2).sum(-1)).mean(-1)
arr_1 = np.array([p1_c[ir, ::s_phi, :] / au, p1_c[ir, ::s_phi, :] / au + scale[:, None, None] * v1_c[ir, ::s_phi, :]])

In [ ]:
X, Y, Z = p1_i.T

vmax = v1_i.max() * 0.1

#C = Normalize()(ri[:, :].T) ## color by radius
C = Normalize(vmin=-vmax, vmax=vmax)(v1_i[:, :, -1].T) ## color by vz

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='3d')
ax.view_init(90, -90)
ax.set_proj_type('ortho')
surf = ax.plot_surface(X / au, Y / au, Z / au, facecolors=plt.cm.RdBu(C), shade=False, linewidth=0, alpha=0.9, rstride=2, cstride=2)

for _ir in range(arr_1.shape[1]):
    for iphi in range(arr_1.shape[2]):
        ax.plot3D(*arr_1[:, _ir, iphi, :].T, 'r', lw=1)

ax.set_xlim(-r_i[-1] / 1.4 / au, r_i[-1] / 1.4 / au)
ax.set_ylim(-r_i[-1] / 1.4 / au, r_i[-1] / 1.4 / au)
ax.set_zlim(-r_i[-1] / 1.4 / au, r_i[-1] / 1.4 / au)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_axis_off()

Make a video

In [ ]:
from pathlib import Path
import subprocess

fpath = Path('frames')
fpath.mkdir(exist_ok=True)

for i in tqdm(np.arange(0, 360)):
    ax.view_init(20.33, i)
    fig.savefig(fpath / f'frame_{i:03d}.png')

p = subprocess.getoutput('ffmpeg -y -framerate 30 -i frames/frame_%03d.png -c:v libx264 -crf 23 -pix_fmt yuv420p video.mp4')

In [ ]:
from IPython.display import Video
Video('video.mp4', width=500, height=500, html_attributes='loop controls autoplay') 

## Now interpolate on sky plane

In [ ]:
_gx = np.linspace(-r_i[-1], r_i[-1], 200)
_gy = np.linspace(-r_i[-1], r_i[-1], 210)
img_xi, img_yi = np.meshgrid(_gx, _gy, indexing='ij')
img_xc = 0.5 * (img_xi[1:, 1:] + img_xi[:-1, 1:])
img_yc = 0.5 * (img_yi[1:, 1:] + img_yi[1:, :-1])

In [ ]:
X, Y, Z = p1_i.T
vxi, vyi, vzi = v1_i.T

values = np.stack((vzi, ri.T)).transpose(1,2,0)

img_z, values_interp = diskwarp.fmodule.interpolate_grid(X, Y, Z, values, img_xc, img_yc)
img_v = values_interp[:, :, 0]
img_r = values_interp[:, :, 1]

Get rid of the values outside of the domain

In [ ]:
img_v[img_v==img_v[0,0]] = np.nan
img_r[img_r==img_r[0,0]] = np.nan

In [ ]:
%matplotlib inline

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10,4), gridspec_kw={'width_ratios': [20, 20, 1]})
axs[1].sharex(axs[0])
axs[1].sharey(axs[0])

c1 = axs[0].contour(img_xc / au, img_yc/ au, img_r / au, 20, colors='k', linewidths=.5)
c2 = axs[0].contourf(img_xc/ au, img_yc/ au, img_r / au, 50, cmap='RdBu')

surf = axs[1].pcolormesh(img_xi/ au, img_yi/ au, img_v / 1e5, cmap='RdBu', vmin=-10, vmax=10)
axs[1].contour(img_xc/ au, img_yc/ au, img_v / 1e5, np.arange(-10, 12, 2.5), colors='k')

fig.colorbar(surf, cax=axs[2])

axs[0].set_title('radius')
axs[1].set_title('radial velocity')

for ax in axs[:2]:
    ax.set_xlim(-100, 100)
    ax.set_ylim(-100, 100)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect(1)

In [ ]:
fig.savefig('warp.pdf', transparent=True, bbox_inches='tight')

## Pyvista

In [ ]:
import pyvista as pv

vpoints = p1_c.reshape(-1, 3)
cloud = pv.PolyData(vpoints)
surf = cloud.delaunay_2d()

surf.save("surf.stl")

In [ ]:
p_full = np.concatenate((p1_c, p1_c[:, 0, :][:, None, :]), axis=1)

In [ ]:
p = pv.Plotter()

In [ ]:
grid = pv.StructuredGrid(*p1_i.T)
s = grid.extract_surface()
s.save('surf2.stl')